In [406]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import joblib

In [407]:
#Functii folosite

#daca nota identificata nu corespunde cu numele fisierului, este identificata gresit
def identificate_gresit(nume_fisiere, y):
    nr = 0                                                  #numarul de note identificate gresit
    linii_gresite = []                                      #append indecsi linii gresite
    linii = nume_fisiere.shape[0]                           #totalul liniilor
    for i in range(linii-1):                                #index pentru fiecare linie (prima linie exclusa automat)
        if nume_fisiere[i][2] == "#":
            nota = nume_fisiere[i][:3]                      #se extrage numele notei de pe linia curenta -> note cu #
        else:
            nota = nume_fisiere[i][:2]                      #se extrage numele notei de pe linia curenta -> note fara #
        if sorted(y[i]) != sorted(nota):                    #cazul: nume = A5# , j = A#5 -> sunt egale
            nr += 1
            linii_gresite.append(i+2)                       #in Excel exista linia de titlu + index-ul incepe de la 1
            print("Nota identificata gresit in Excel pe randul {}, nota adevarata {}, nota falsa {}.".format(i+2, nota, y[i]))
    print("\nNumar note identificate gresit: {}.".format(nr))
    
    return linii_gresite, nr, y


#label-urile identificate gresit se schimba cu cele date de numele fisierului
def schimbare_label(linii_gresite, nume_fisiere, y):
    for i in linii_gresite:
        if nume_fisiere[i-2][2] == "#":
            y[i-2] = nume_fisiere[i-2][:3]
        else:
            y[i-2] = nume_fisiere[i-2][:2]
    print("\nS-au schimbat label-urile. Verificam daca mai sunt label-uri gresite...")
    print("\nLabel-uri dupa schimbare:\n", y)

    return identificate_gresit(nume_fisiere, y)


#se sterge greseala din fiecare variabila(nume_fisier, frecvente, note)
def stergere_gresite(nume_fisiere, X, y, linii_gresite):
    # randul din excel este cu 2 mai mare(1 pt randul de titlu + 1 pt ca index-ul incepe de la 0)
    nume_fisiere = np.delete(nume_fisiere, [i - 2 for i in linii_gresite], axis=0)
    X = np.delete(X, [i - 2 for i in linii_gresite], axis=0)
    y = np.delete(y, [i - 2 for i in linii_gresite], axis=0)

    return nume_fisiere, X, y

In [408]:
df_train = pd.read_csv('autocor_train.csv')    #dataframe
nume_fisiere = df_train.iloc[:, 0].values
frecvente = df_train.iloc[:, 1].values
note = df_train.iloc[:, 2].values

X_train = frecvente    #features
y_train = note         #labels

In [409]:
linii_gresite, nr, y_train = identificate_gresit(nume_fisiere, y_train)

Nota identificata gresit in Excel pe randul 3, nota adevarata A3#, nota falsa A#2.
Nota identificata gresit in Excel pe randul 7, nota adevarata A3, nota falsa A2.
Nota identificata gresit in Excel pe randul 16, nota adevarata A4, nota falsa A2.
Nota identificata gresit in Excel pe randul 20, nota adevarata A5#, nota falsa D#4.
Nota identificata gresit in Excel pe randul 29, nota adevarata B3, nota falsa B2.
Nota identificata gresit in Excel pe randul 33, nota adevarata B4, nota falsa G2.
Nota identificata gresit in Excel pe randul 50, nota adevarata C4#, nota falsa C#3.
Nota identificata gresit in Excel pe randul 55, nota adevarata C4, nota falsa C3.
Nota identificata gresit in Excel pe randul 80, nota adevarata D4#, nota falsa G#2.
Nota identificata gresit in Excel pe randul 101, nota adevarata E4, nota falsa E3.
Nota identificata gresit in Excel pe randul 111, nota adevarata F3#, nota falsa F#2.
Nota identificata gresit in Excel pe randul 115, nota adevarata F3, nota falsa F2.
Nota 

In [410]:
linii_gresite, nr, y_train = schimbare_label(linii_gresite, nume_fisiere, y_train)


S-au schimbat label-urile. Verificam daca mai sunt label-uri gresite...

Label-uri dupa schimbare:
 ['A#3' 'A3#' 'A#3' 'A#3' 'A3' 'A3' 'A3' 'A3' 'A#4' 'A#4' 'A#4' 'A#4' 'A#4'
 'A4' 'A4' 'A4' 'A4' 'A4' 'A5#' 'A#5' 'A#5' 'A#5' 'A5' 'A5' 'A5' 'A5' 'B3'
 'B3' 'B3' 'B3' 'B4' 'B4' 'B4' 'B4' 'B4' 'B5' 'B5' 'B5' 'B5' 'C#3' 'C#3'
 'C#3' 'C#3' 'C3' 'C3' 'C3' 'C3' 'C#4' 'C4#' 'C#4' 'C#4' 'C#4' 'C4' 'C4'
 'C4' 'C4' 'C4' 'C#5' 'C#5' 'C#5' 'C#5' 'C5' 'C5' 'C5' 'C5' 'C6' 'C6' 'C6'
 'C6' 'D#3' 'D#3' 'D#3' 'D#3' 'D3' 'D3' 'D3' 'D3' 'D#4' 'D4#' 'D#4' 'D#4'
 'D#4' 'D4' 'D4' 'D4' 'D4' 'D4' 'D#5' 'D#5' 'D#5' 'D#5' 'D5' 'D5' 'D5'
 'D5' 'E3' 'E3' 'E3' 'E3' 'E4' 'E4' 'E4' 'E4' 'E4' 'E5' 'E5' 'E5' 'E5'
 'F#3' 'F3#' 'F#3' 'F#3' 'F3' 'F3' 'F3' 'F3' 'F#4' 'F4#' 'F#4' 'F#4' 'F#4'
 'F4' 'F4' 'F4' 'F4' 'F4' 'F#5' 'F#5' 'F#5' 'F#5' 'F5' 'F5' 'F5' 'F5'
 'G#3' 'G#3' 'G#3' 'G#3' 'G3' 'G3' 'G3' 'G3' 'G#4' 'G#4' 'G#4' 'G#4' 'G#4'
 'G4' 'G4' 'G4' 'G4' 'G4' 'G#5' 'G#5' 'G#5' 'G#5' 'G5' 'G5' 'G5' 'G5']

Numar note identific

In [411]:
X_train.reshape(-1,1)

array([[ 234.57446809],
       [ 116.66666667],
       [ 234.57446809],
       [ 234.57446809],
       [ 220.5       ],
       [ 110.25      ],
       [ 220.5       ],
       [ 222.72727273],
       [ 469.14893617],
       [ 469.14893617],
       [ 469.14893617],
       [ 469.14893617],
       [ 479.34782609],
       [ 441.        ],
       [ 110.25      ],
       [ 441.        ],
       [ 441.        ],
       [ 432.35294118],
       [ 310.56338028],
       [ 918.75      ],
       [ 918.75      ],
       [ 918.75      ],
       [ 882.        ],
       [ 882.        ],
       [ 882.        ],
       [ 882.        ],
       [ 247.75280899],
       [ 123.87640449],
       [ 247.75280899],
       [ 247.75280899],
       [ 501.13636364],
       [  98.87892377],
       [ 501.13636364],
       [ 501.13636364],
       [ 490.        ],
       [1002.27272727],
       [1002.27272727],
       [1002.27272727],
       [1002.27272727],
       [ 138.67924528],
       [ 138.67924528],
       [ 138.679

In [412]:
y_train.reshape(-1,1)

array([['A#3'],
       ['A3#'],
       ['A#3'],
       ['A#3'],
       ['A3'],
       ['A3'],
       ['A3'],
       ['A3'],
       ['A#4'],
       ['A#4'],
       ['A#4'],
       ['A#4'],
       ['A#4'],
       ['A4'],
       ['A4'],
       ['A4'],
       ['A4'],
       ['A4'],
       ['A5#'],
       ['A#5'],
       ['A#5'],
       ['A#5'],
       ['A5'],
       ['A5'],
       ['A5'],
       ['A5'],
       ['B3'],
       ['B3'],
       ['B3'],
       ['B3'],
       ['B4'],
       ['B4'],
       ['B4'],
       ['B4'],
       ['B4'],
       ['B5'],
       ['B5'],
       ['B5'],
       ['B5'],
       ['C#3'],
       ['C#3'],
       ['C#3'],
       ['C#3'],
       ['C3'],
       ['C3'],
       ['C3'],
       ['C3'],
       ['C#4'],
       ['C4#'],
       ['C#4'],
       ['C#4'],
       ['C#4'],
       ['C4'],
       ['C4'],
       ['C4'],
       ['C4'],
       ['C4'],
       ['C#5'],
       ['C#5'],
       ['C#5'],
       ['C#5'],
       ['C5'],
       ['C5'],
       ['C5'],
       ['C5'],

In [413]:
# K Nearest Neighbor
# 3 parameters in the model creation:
# 1. n_neighbors = 5 neighborhood point required for classifying a point
# 2. distance metric = Minkowski, check equation
# 3. p = 2, Euclidean distance, parameter in Minkowski equation

model = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
X_train = X_train.reshape(-1,1)
model.fit(X_train, y_train)

KNeighborsClassifier()

In [414]:
joblib.dump(model, 'knn_autocor_individual.joblib')      #model salvat

['knn_autocor_individual.joblib']

In [415]:
df_test = pd.read_csv('autocor_test.csv')

nume_fisiere = df_test.iloc[:, 0].values
frecvente = df_test.iloc[:, 1].values
note = df_test.iloc[:, 2].values

X_test = frecvente
y_test = note

In [416]:
linii_gresite, nr, y_test = identificate_gresit(nume_fisiere, y_test)

Nota identificata gresit in Excel pe randul 29, nota adevarata A5#, nota falsa A#4.
Nota identificata gresit in Excel pe randul 31, nota adevarata A5#, nota falsa A#4.
Nota identificata gresit in Excel pe randul 32, nota adevarata A5#, nota falsa A#4.
Nota identificata gresit in Excel pe randul 33, nota adevarata A5#, nota falsa D#4.
Nota identificata gresit in Excel pe randul 76, nota adevarata C4#, nota falsa D#4.
Nota identificata gresit in Excel pe randul 133, nota adevarata D5#, nota falsa D#4.
Nota identificata gresit in Excel pe randul 139, nota adevarata D5, nota falsa D4.
Nota identificata gresit in Excel pe randul 147, nota adevarata E4, nota falsa E3.
Nota identificata gresit in Excel pe randul 149, nota adevarata E4, nota falsa E3.
Nota identificata gresit in Excel pe randul 176, nota adevarata F4#, nota falsa F#3.
Nota identificata gresit in Excel pe randul 184, nota adevarata F4, nota falsa E4.
Nota identificata gresit in Excel pe randul 196, nota adevarata F5, nota falsa

In [417]:
linii_gresite, nr, y_test = schimbare_label(linii_gresite, nume_fisiere, y_test)


S-au schimbat label-urile. Verificam daca mai sunt label-uri gresite...

Label-uri dupa schimbare:
 ['A#3' 'A#3' 'A#3' 'A#3' 'A#3' 'A#3' 'A3' 'A3' 'A3' 'A3' 'A3' 'A3' 'A#4'
 'A#4' 'A#4' 'A#4' 'A#4' 'A#4' 'A#4' 'A4' 'A4' 'A4' 'A4' 'A4' 'A4' 'A4'
 'A#5' 'A5#' 'A#5' 'A5#' 'A5#' 'A5#' 'A5' 'A5' 'A5' 'A5' 'A5' 'A5' 'B3'
 'B3' 'B3' 'B3' 'B3' 'B3' 'B4' 'B4' 'B4' 'B4' 'B4' 'B4' 'B5' 'B5' 'B5'
 'B5' 'B5' 'B5' 'C#3' 'C#3' 'C#3' 'C#3' 'C#3' 'C#3' 'C3' 'C3' 'C3' 'C3'
 'C3' 'C3' 'C#4' 'C#4' 'C#4' 'C#4' 'C#4' 'C#4' 'C4#' 'C4' 'C4' 'C4' 'C4'
 'C4' 'C4' 'C4' 'C#5' 'C#5' 'C#5' 'C#5' 'C#5' 'C#5' 'C5' 'C5' 'C5' 'C5'
 'C5' 'C5' 'C6' 'C6' 'C6' 'C6' 'C6' 'C6' 'D#3' 'D#3' 'D#3' 'D#3' 'D#3'
 'D#3' 'D3' 'D3' 'D3' 'D3' 'D3' 'D3' 'D#4' 'D#4' 'D#4' 'D#4' 'D#4' 'D#4'
 'D#4' 'D4' 'D4' 'D4' 'D4' 'D4' 'D4' 'D4' 'D#5' 'D#5' 'D#5' 'D#5' 'D#5'
 'D5#' 'D5' 'D5' 'D5' 'D5' 'D5' 'D5' 'E3' 'E3' 'E3' 'E3' 'E3' 'E3' 'E4'
 'E4' 'E4' 'E4' 'E4' 'E4' 'E4' 'E5' 'E5' 'E5' 'E5' 'E5' 'E5' 'F#3' 'F#3'
 'F#3' 'F#3' 'F#3' 'F#3' 'F3' 'F3

In [418]:
model = joblib.load('knn_autocor_individual.joblib')     #model incarcat

X_test = X_test.reshape(-1,1)
predictii = model.predict(X_test)

In [419]:
score = accuracy_score(y_test, predictii)
score       #eroare data de notele gresite din test si de cateva note gresite din modelul de train

0.9141630901287554

In [420]:
cm = confusion_matrix(y_test, predictii)
cm

array([[6, 0, 0, ..., 0, 0, 0],
       [0, 7, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 7, 0, 0],
       [0, 0, 0, ..., 0, 6, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [421]:
#acum un test cu stergerea liniilor gresite din test

In [422]:
df_test = pd.read_csv('autocor_test.csv')

nume_fisiere = df_test.iloc[:, 0].values
frecvente = df_test.iloc[:, 1].values
note = df_test.iloc[:, 2].values

X_test = frecvente
y_test = note

linii_gresite, nr, y_test = identificate_gresit(nume_fisiere, y_test)

Nota identificata gresit in Excel pe randul 29, nota adevarata A5#, nota falsa A#4.
Nota identificata gresit in Excel pe randul 31, nota adevarata A5#, nota falsa A#4.
Nota identificata gresit in Excel pe randul 32, nota adevarata A5#, nota falsa A#4.
Nota identificata gresit in Excel pe randul 33, nota adevarata A5#, nota falsa D#4.
Nota identificata gresit in Excel pe randul 76, nota adevarata C4#, nota falsa D#4.
Nota identificata gresit in Excel pe randul 133, nota adevarata D5#, nota falsa D#4.
Nota identificata gresit in Excel pe randul 139, nota adevarata D5, nota falsa D4.
Nota identificata gresit in Excel pe randul 147, nota adevarata E4, nota falsa E3.
Nota identificata gresit in Excel pe randul 149, nota adevarata E4, nota falsa E3.
Nota identificata gresit in Excel pe randul 176, nota adevarata F4#, nota falsa F#3.
Nota identificata gresit in Excel pe randul 184, nota adevarata F4, nota falsa E4.
Nota identificata gresit in Excel pe randul 196, nota adevarata F5, nota falsa

In [423]:
nume_fisiere, X_test, y_test = stergere_gresite(nume_fisiere, X_test, y_test, linii_gresite)

In [424]:
X_test = X_test.reshape(-1,1)
predictii = model.predict(X_test)

In [425]:
score = accuracy_score(y_test, predictii)
score         #accuracy mai mare, doar eroare de train

0.9861111111111112

In [426]:
cm = confusion_matrix(y_test, predictii)
cm

array([[6, 0, 0, ..., 0, 0, 0],
       [0, 7, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 6, 0, 0],
       [0, 0, 0, ..., 0, 7, 0],
       [0, 0, 0, ..., 0, 0, 6]], dtype=int64)

In [427]:
#daca se scade numarul de vecini din model, se observa o usoara crestere a acuratetii